In [21]:
import json
import openai
import requests
from tenacity import retry, wait_random_exponential, stop_after_attempt
from termcolor import colored

GPT_MODEL = "gpt-4"

In [2]:
@retry(wait=wait_random_exponential(multiplier=1, max=40), stop=stop_after_attempt(3))
def chat_completion_request(messages, functions=None, function_call=None, model=GPT_MODEL):
    headers = {
        "Content-Type": "application/json",
        "Authorization": "Bearer " + openai.api_key,
    }
    json_data = {"model": model, "messages": messages}
    if functions is not None:
        json_data.update({"functions": functions})
    if function_call is not None:
        json_data.update({"function_call": function_call})
    try:
        response = requests.post(
            "https://api.openai.com/v1/chat/completions",
            headers=headers,
            json=json_data,
        )
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e

In [45]:
def multiply (*args):
    rc = 1
    for i in args:
        rc = rc *i
    return rc

def add (*args):
    return sum(args)

available_functions = {
    "multiply":multiply,
    "add":add,
}

functions = [
    {
        "name" : "multiply",
        "description": "Multiply arguments together",
        "parameters" : {
            "type": "object",
            "properties" : {
                "input": {
                     "type":"array",
                     "items": {
                        "type":"number",
                         "minLength":2
                     }
                } 
              }
        }
    },
    {
        "name" : "add",
        "description": "Add arguments together",
        "parameters" : {
            "type": "object",
            "properties" : {
                "input": {
                     "type":"array",
                     "items": {
                        "type":"number",
                         "minLength":2
                     }
                } 
              }
        }
    }
]
        

In [42]:
messages = [
 {
     "role":"system",
     "content":"Answer user questions returning function arguments"
 },
 {
     "role": "user",
     "content":"What is five plus 34 + one hundred"
 }
]

In [46]:
resp = chat_completion_request(messages, functions)

In [47]:
response_message = resp.json()['choices'][0]['message']
print(response_message)

{'role': 'assistant', 'content': None, 'function_call': {'name': 'add', 'arguments': '{\n  "input": [5, 34, 100]\n}'}}


In [48]:
if 'function_call' in response_message and response_message['function_call'] is not None:
    to_call = response_message['function_call']['name']
    f_args = json.loads(response_message['function_call']['arguments'])['input']
    result = available_functions[to_call](*f_args)
    print(result)
else:
    print(f'no function was returned - {response_message["content"]}')
    

139
